# Importanto Bibliotecas & Definindo Constantes

In [ ]:
from typing import Dict, List
import os
import re
from collections import Counter

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline

In [ ]:
DATA_INPUT_FOLDER = 'data'
ARCHIVE_FOLDER = 'archive'
OUTPUT_FOLDER = 'output'

# Importando os Dados

Extraídos da base: https://www.kaggle.com/datasets/patkle/metacritic-scores-for-games-movies-tv-and-music

Os dados são referentes a notas de jogos, filmes, músicas e séries (de TV) avaliados por usuários e pelo próprio domínio Metacritic.

In [ ]:
# Um dicionário contendo os 4 DataFrames, separados 
df_dict : Dict[ str, pd.DataFrame ] = {}
for file_data in os.listdir(DATA_INPUT_FOLDER):
    df_dict[file_data[:-4]] = pd.read_csv(
        os.path.join( DATA_INPUT_FOLDER, file_data ),
        index_col=0,
        parse_dates= ['release_date'],
        dayfirst= True
    )

print(df_dict.keys())

# Analizando Inicialmente a Base exclusiva de Jogos

## Visão Geral

In [ ]:
games_df = df_dict['games']

# Observar informações gerais sobre os dados e tipos
display(games_df.info())
display(games_df.head())

O banco apresenta 20022 registros (linhas), com 7 propriedades (colunas):

- metascore: nota da plataforma
- plataforma
- data de lançamento
- número de ordenação (não está claro a que se refere)
- resumo: descrição breve do jogo
- título: nome do jogo
- nota dos usuários

Podemos verificar que 

- metascore é de 0 a 100 e está ocupando espaco de um int 64
- Há dados nulos (faltando) para o resumo (summary) de alguns registros
- Os dados de nota dos usuários (user_score) estão sendo tratados como texto (object) em vez de números (float)

## Entendendo o Significado dos Dados

### sort_no

Analisar número de ordenação (sort_no), para tentar entendê-lo e descobrir se ele é útil

In [ ]:
# Conferir se é único
np.any(games_df['sort_no'].duplicated())

In [ ]:
# Conferir se tem alguma relação com outra coluna, principalmente de notas
games_df.sort_values('sort_no')

In [ ]:
# comparar se a ordenação pelo metascore (decrescente) e pelo sort_no (crescente) tem mesmas notas metascore
np.all(
    games_df.sort_values('metascore', ascending= False, ignore_index= True)['metascore'] == 
    games_df.sort_values('sort_no', ignore_index= True)['metascore']
)

Logo concluímos que sort_no é uma ordenação de ranking com base nos maiores metascore

## Levantando Hipóteses e Questionamentos

1. Qual o top 10 jogos mais bem avaliados pelo site? E pelos usuários?
2. Qual a plataforma que mais aparece entre os 100 melhores avaliados pelos usuários?
3. As notas dos jogos melhoraram a cada ano? E para cada plataforma ao longo dos anos?
4. Tem alguma epoca do ano que apresenta maior sucesso em relação a notas maiores?
5. Que palavras mais aparecem nos títulos dos 1000 melhores jogos? E dos 100 piores? E nos resumos?
6. Histograma - Distribuição de Lançamento de Jogos por Ano
7. Histograma - Distribuição de Lançamento de Jogos por Mês
8. Histograma - Distribuição da avaliação dos usuários
9. Histograma - Distribuição da avaliação do Metacritic
10. Heatmap - Correlação entre as variáveis
11. Qual o número de games produzidos para cada plataforma do Dataset? Esses valores indicam qual porcentagem do total?
12. Comparativo: PlayStation 4 e PlayStation 5
13. Games Disponíveis por Mês por 7 anos (PS4 e Previsão PS5)
14. Qual a proporção de cada tipo de classificação atribuída aos games? Qual a porcentagem do total de cada uma?
15. Distribuição de Classificações Metacritic
16. Quais são os top 10 games mais bem avaliados pelos usuários e pelo metacritic? Eles são muito diferentes?
17. Quais são os bottom 10 games pior avaliados pelos usuários e pelo metacritic? Eles são muito diferentes?

## Limpando dados

### metascore (número de bytes)

In [ ]:
# int 8 vai até 128, contemplando todos os valores
games_df = ( games_df.astype({'metascore': 'int8'}) )

### user_score

In [ ]:
# tentativa de converter o tipo dos dados de user_score, para ver os valores que falham
set_errors = set()
for score in games_df['user_score']:
    try:
        float(score)
    except Exception as e:
        set_errors.add(str(e))
print(set_errors)

In [ ]:
# Entendendo os registros com nota tbd (To Be Determined, traduzido como "a ser determinado")
display( games_df[ games_df['user_score'] == 'tbd' ].head() )
print(f"Número de registros com nota dos usuários pendente: { np.sum( games_df['user_score'] == 'tbd' ) }")
print(f"Porcentagem de registros com nota dos usuários pendente: { np.sum( games_df['user_score'] == 'tbd' ) / len( games_df ) :0.1%}")

Uma possível explicação para esse valor é de que esse número é uma média e ainda não tiveram avaliações de usuários o suficiente para computar uma média adequada.

Podemos adotar algumas abordagens:

- substituir os valores pela média geral
- separar em grupos de acordo com algum critério e substituir os valores pelas médias de cada grupo 
    - mesmo metascore
    - mesma plataforma
    - mesmo metascore e plataforma
- substituir os valores usando técnicas que mantenham a distribuição dos dados válidos
    - bfill (backward fill) : substituir pela proxima observacao
    - ffill (forward fill) : substituir pela observacao anterior
- excluir linhas

Como esses dados representam 7% dos valores, não desejo excluí-los, então tentarei uma média de acordo o agrupamento pela plataforma

In [ ]:
# para comparação
games_df['user_score_raw'] = games_df['user_score']

# Transformar a coluna em float, e os valores faltantes em NaN para poder computar a média
games_df['user_score'] = games_df['user_score'].replace('tbd', np.nan).astype(float)

# Criar uma cópia do dataframe, para alterar o user_score somente na cópia
games_df['user_score'] = games_df['user_score'].fillna(
    games_df.groupby('platform')['user_score'].transform('mean')
)

display( games_df[['user_score', 'user_score_raw']].head() )
display( games_df[games_df['user_score_raw'] == 'tbd'][['platform', 'user_score', 'user_score_raw']].head(10) )

games_df = games_df.drop(columns= 'user_score_raw')

### summary

In [ ]:
print('número de resumos faltantes:', games_df['summary'].isna().sum() )
display( games_df[games_df['summary'].isna()].head() )

Parece ser um caso de dados não coletados, pois a falta de um texto no csv gera um NaN

Podemos adotar algumas abordagens:

- procurar os dados na internet e preencher manualmente
- substituir por um valor padrão
- deixar como está
- excluir linhas

Como é uma descrição do jogo, não há necessidade em perder os outros dados por conta dele.

Substituirei os valores por uma string vazia, assim como está em games.csv

In [ ]:
games_df['summary'] = games_df['summary'].fillna('')
games_df['summary'].isna().sum()

## Criando Novas Variáveis

### Trimestre de Lançamento

Como forma de expandir nossas possibilidades de análise, vamos inserir nos dados uma nova variável que informa qual o trimestre do ano em que o Jogo foi lançado.

In [ ]:
def set_year_quarter(data: pd.DataFrame) -> pd.DataFrame:
    '''Retorna uma nova variável que informa o trimestre do ano em que
    o jogo foi lançado. Essa definição de trimestre se baseia no mês de lançamento
    da coluna release_date.
    
    PARÂMETROS:
    
        data > DataFrame para se aplicar a alteração.
    '''
    data['release_quarter'] = data['release_date'].apply(
        lambda value:
            'Q1' if value.month in [1, 2, 3] else
            'Q2' if value.month in [4, 5, 6] else
            'Q3' if value.month in [7, 8, 9] else
            'Q4' if value.month in [10, 11, 12] else
            value
            )
    
    return data

In [ ]:
# Aplicando função:
games_df = set_year_quarter(games_df)

# Checando resultado:
games_df.sample(10)

### Subdivisões da Variável release_date

Com a nova variável de trimestre de lançamento criada, vamos retornar as atenções a variável 'release_date' e utilizá-la para criar três variáveis distintas que informam apenas o dia, mês e ano do lançamento.

Essa divisão da variável 'release_date' será feita para facilitar a criação de plots na etapa de análises.

- Criando variável 'release_day':

In [ ]:
# Criando nova variável:
games_df['release_day'] = games_df['release_date'].dt.day

- Criando variável 'release_month':

In [ ]:
games_df['release_month'] = games_df['release_date'].dt.month

- Criando variável 'release_year':

In [ ]:
# Criando nova variável:
games_df['release_year'] = games_df['release_date'].dt.year

In [ ]:
games_df.head(5)

### Metascore_class

Essa variável tem como objetivo abrigar a classificação dos valores do Metascore. De acordo com um [FAQ do próprio site](https://www.metacritic.com/about-metascores), para cada range de notas temos uma classificação geral atribuída a ele como ilustrado na imagem abaixo retirada diretamente do FAQ:

<img src="https://raw.githubusercontent.com/JoSEPHDev2022/Metacritc_Scores_Video_Games/main/images/metascore_classification.png" width=700 height=350>

https://www.metacritic.com/about-metascores

In [ ]:
def set_metascore_class(data: pd.DataFrame) -> pd.DataFrame:
    '''Retorna uma nova variável que consta a classificação do Jogo
    com base em sua nota. Essas classificações são:
    
        - "Universal Acclaim" para games com notas entre 90 e 100;
        - "Generally Favorable Reviews" para games com notas entre 75 e 89;
        - "Mixed or Average Reviews" para games com notas entre 50 e 74;
        - "Generally Unfavorable Reviews" para games com notas entre 20 e 49;
        - "Overwhelming Dislike" para games com notas entre 0 e 19.

        PARÂMETROS:
        
        data > DataFrame para se aplicar a alteração. 
    '''
    data['classification'] = data['metascore'].apply(
        lambda value:
            'Universal Acclaim' if 90 <= value <= 100 else
            'Generally Favorable' if 75 <= value <= 89 else
            'Mixed or Average' if 50 <= value <= 74 else
            'Generally Unfavorable' if 20 <= value <= 49 else
            'Overwhelming Dislike'   
            )
    
    return data

In [ ]:
# Aplicando função:
games_df = set_metascore_class(games_df)

In [ ]:
# Visualizando resultados:
games_df.sample(10)

## Funcoes auxiliares

Com objetivo de estudar palavras presentes na descrição e título, criarei uma função que realiza a extração das palavras

In [ ]:
def extract_words(texto : str) -> List[str]:
    return re.findall('\w+', texto)

extract_words('Injustice 2: Legendary Edition')

In [ ]:
def count_words(serie_texto : pd.Series) -> Dict[str, int]:
    '''
    It counts the number of text elements from the input series which contain a certain word
    @input:
        - serie_texto: a string-type pd.Series
    @output:
        A dict where each key is a word and the value its count as described
    '''
    word_counter = Counter()
    for texto in serie_texto:
        word_counter.update( Counter( set( extract_words(texto) ) ) )
    return word_counter

count_words(pd.Series(['Injustice 2: Legendary Edition', 'Injustice 2: Legendary Edition', 'Mini Metro']))

## EDA

### Análise Univariada

In [ ]:
games_df.columns

#### Ánalise de Variável Categórica: platform

In [ ]:
games_df['platform'].value_counts(ascending= True).plot.barh()

Há muitas plataformas, e muitas estão relacionadas pelo 'fabricante', entao criaremos outra coluna

In [ ]:
platform_manufacturer = {
    'PC' : 'PC',
    'PlayStation' : 'PlayStation',
    'PlayStation 5' : 'PlayStation',
    'PlayStation 4' : 'PlayStation',
    'PlayStation 3' : 'PlayStation',
    'PlayStation 2' : 'PlayStation',
    'PlayStation Vita' : 'PlayStation',
    'PSP' : 'PlayStation',
    'Nintendo 64' : 'Nintendo',
    'GameCube' : 'Nintendo',
    'Wii U' : 'Nintendo',
    'Wii' : 'Nintendo',
    'Switch' : 'Nintendo',
    'Game Boy Advance' : 'Nintendo',
    'DS' : 'Nintendo',
    '3DS' : 'Nintendo',
    'Xbox Series X' : 'Xbox',
    'Xbox' : 'Xbox',
    'Xbox One' : 'Xbox',
    'Xbox 360' : 'Xbox',
    'Xbox 360' : 'Xbox',
    'Stadia' : 'Other',
    'Dreamcast' : 'Other'
}
games_df['manufacturer'] = games_df['platform'].map(platform_manufacturer)
games_df['manufacturer'].value_counts().plot.bar(rot= 45)

Assim podemos ver que a PlayStation na verdade ultrapassa o número de jogos de PC no total,
considerando que ela renova o console de tempos em tempos

#### Histogramas: analisando variáveis numéricas continuas

##### funcao auxiliar

In [ ]:
# Definindo uma função para plotagem dos histogramas pelo MatPlotLib:

def histogram_plot(data: pd.DataFrame, column: pd.Series, bin_width: float, title: str):
    '''Retorna um Histograma detalhado sobre os dados fornecidos.
    
    PARÂMETROS:

        - data > Um DataFrame do Pandas contendo os dados;
        - column > Uma Series do Pandas contendo os dados desejados para plotagem;
        - bin_width: > Número float para a dimensão das barras do histograma;
        - title > O título desejado para o plot.
    '''
    fig, ax1 = plt.subplots(figsize=(12,6))

    ax2 = sns.histplot(data=data, 
                       x=column, 
                       kde=True,
                       label='Frequência', 
                       binwidth=bin_width,
                       edgecolor='black', 
                       line_kws={'linestyle': 'dashdot'},
                       color='#151bbf')
    
    ax2.set_title(title, fontdict={'size': 15, 
                                   'weight': 'bold'})
    
    ax2.set_ylabel('Num. Registros', fontdict={'size': 12,
                                               'family': 'monospace'})

    ax2.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda number, char: format(int(number), ',')))

    ax2.set_xlabel('')

##### release_year

In [ ]:
# Histograma:
histogram_plot(games_df, 
               'release_year', 
               1, 
               'Distribuição de Lançamento de Jogos por Ano')

# Anotando lançamento PS2:
plt.annotate('Lançamento PS2', 
             fontsize=9, 
             fontfamily='monospace',
             xy=(2000.1, 370), 
             xytext=(1994.5,400), 
             arrowprops={'arrowstyle':'->', 'color':'black'},
             bbox={'boxstyle': 'round', 'fc': '0.65', 'color': '#0bc3d4'})

# Anotando lançamento X-Box:
plt.annotate('Lançamento X-Box', 
             fontsize=9, 
             fontfamily='monospace',
             xy=(2001.1, 555), 
             xytext=(1994.5,570), 
             arrowprops={'arrowstyle':'->', 'color':'black'},
             bbox={'boxstyle': 'round', 'fc': '0.65', 'color': '#0bd422'})

plt.show()

Centralidade

- Em relação aos anos de lançamento dos jogos, temos uma tendência central entre 2015 e 2020, onde as maiores quantidades de data points se acumulam.

Amplitude

In [ ]:
amplitude = games_df['release_year'].max() - games_df['release_year'].min()

print(f'Período de {amplitude} anos.')

##### release_month

In [ ]:
# Histograma:
histogram_plot(games_df, 
               'release_month',
               0.95,
               'Distribuição de Lançamento de Jogos por Mês')

Centralidade

- Em relação aos meses de lançamento dos jogos, temos uma tendência central mais ao final do ano, a partir de Setembro (9) a Novembro (11).

Amplitude

In [ ]:
amplitude = games_df['release_month'].max() - games_df['release_month'].min() 

print(f'Intervalo de Tempo de {amplitude} meses.')

##### user_score

In [ ]:
# Histograma:
histogram_plot(games_df, 
               'user_score',
               0.3,
               'Distribuição de Avaliações de Usuários')

Centralidade

- Em relação as notas fornecidas pelos usuários, temos uma tendência central dos dados entre 7 e 8, como a mediana de 7.1 nos apontou anteriormente.

Amplitude

- Temos uma amplitude pequena por se tratar de uma avaliação, indo de 0 a 10.

##### metascore

In [ ]:
# Histograma:
histogram_plot(games_df, 
               'metascore',
               1,
               'Distribuição de Avaliações Metacritic')

Centralidade

- Em relação as notas fornecidas pelo Metacritic, temos uma tendência central dos dados entre 70 e 80. A distribuição de notas do Metacritic parece seguir o mesmo padrão das notas fornecidas pelos usuários, o que demonstra que as notas fornecidas pelo site não são tão discrepantes em relação as fornecidas pelos usuários. A correlação exata dessas duas métricas de avaliação será estudada mais a frente.

Amplitude

- Temos uma amplitude de dados variando de 0 a 100.

In [ ]:
_= games_df['metascore'].plot.box(vert= False)

metascore apresentam muitos outliers a esquerda (valores baixos)
e mais da metade dos jogos está avaliado no intervalo de 60 a 80.

### Análise Bivariada

#### Correlação entre as variáveis

In [ ]:
# Criando matriz de correlações:

correlation_matrix = games_df.select_dtypes(include=[np.number]).corr()

correlation_matrix

In [ ]:
# Selecionando as colunas relevantes (excluindo 'sort_no')
cols = [col for col in games_df.columns if col != 'sort_no']
subset_df = games_df[cols]

# Calculando a matriz de correlação
correlation_matrix = subset_df.corr()

# Criando o Heatmap:
fig, ax = plt.subplots(figsize=(16, 6))

ax = sns.heatmap(correlation_matrix, 
                 vmin=-1, 
                 vmax=1, 
                 annot=True, 
                 linewidths=1, 
                 cmap='coolwarm')

# Formatação:
plt.title('Correlações de Variáveis', fontdict={'size': 18, 'weight': 'bold'}, pad=16)

plt.show()

<font color='#039c0b'>**Correlação igual a 1**</font>

- Correlação positiva perfeita. Quando o valor de uma variável X aumenta, o valor de outra variável Y também aumenta.

<font color='#039c0b'>**Correlação acima de 0**</font>

- Correlação positiva. Quando o valor de uma variável X aumenta, o valor de outra variável Y tende a aumentar, porém essa regra não é absoluta, tendo valores da variável Y que não se alteram ou diminuem.

**Correlação igual a 0**

- Não existe correlação entre as duas variáveis.

<font color='#a80505'>**Correlação menor que 0**</font>

- Correlação negativa. Quando o valor de uma variável X aumenta, o valor de outra variável Y tende a diminuir, porém essa regra não é absoluta, tendo valores da variável Y que não se alteram ou aumentam.

<font color='#a80505'>**Correlação igual a -1**</font>

- Correlação neagtiva perfeita. Quando o valor de uma variável X aumenta, o valor de outra variável Y diminui.

#### user_score e metascore

In [ ]:
def scale_scores(row : pd.DataFrame) -> pd.DataFrame:
    row['metascore'] /= 100
    row['user_score'] /= 10
    return row

_= sns.violinplot( games_df[['user_score', 'metascore']].apply(scale_scores, axis= 1) )